1) A representação de uma determinada mensagem digital ternária, isto é formada por três bits,
forma um cubo cujos vértices correspondem a mesma representação digital. Supondo que ao
transmitirmos esta mensagem a mesma possa ser contaminada por ruído formado em torno de
cada vértice uma nuvem esférica de valores aleatórios com raio máximo é 0.1. Formule este
problema como um problema de classificação de padrões e treine uma rede Perceptron de
Rosenblatt (Perceptron de camada única) para atuar como classificador/decodificador. Para
solução do problema defina antes um conjunto de treinamento e um conjunto de validação.
Dica: O problema pode ser formulado como um problema de classificação de 8 padrões
diferentes, sendo que cada padrão representa um vértice do cubo.
Padrão 1: x = {0,0,0} com vetor resposta d = {1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0}
Padrão 2: x = {0,0,1} com vetor resposta d = {-1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0}
Padrão 3: x = {0,1,0} com vetor resposta d = {-1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0}
Padrão 4: x = {0,1,1} com vetor resposta d = {-1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0}
Padrão 5: x = {1,0,0} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0}
Padrão 6: x = {1,0,1} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0, -1.0}
Padrão 7: x = {1,1,0} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, -1.0}
Padrão 8: x = {1,1,1} com vetor resposta d = {-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0}

In [ ]:

import random
import numpy as np
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 
from keras.activations import hard_sigmoid
from keras.optimizers import SGD

def one_hot_encode(x):
    # one_hot = np.empty(shape=(len(x),8))
    # one_hot.fill(-1)
    one_hot = np.zeros([len(x), 8])
    for i in enumerate(x):
        one_hot[(i[0], i[1])] = 1
    return one_hot

def rand():
    return round(random.randint(0,1) + random.uniform(-0.1, 0.1),4)

total_data = 1000
data_X = np.zeros((total_data,3), dtype=np.float32)
data_y = np.zeros(total_data, dtype=np.int8)

for i in range(total_data):
    val = np.array([rand(), rand(), rand()])
    data_X[i] = val

    code = np.array([round(val[0]), round(val[1]), round(val[2])])
    label = np.array([code[0]*4 + code[1]*2 + code[2]])
    data_y[i] = label

data_y = one_hot_encode(data_y)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(data_X, data_y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)
print(y_train.shape); print(y_test.shape)

(700, 3)
(300, 3)
(700, 8)
(300, 8)


In [ ]:
model = Sequential()

model.add(Dense(8, activation='relu'))
model.add(Dense(8, input_dim=3, activation='hard_sigmoid'))

opt = SGD(learning_rate=0.5)
model.compile(optimizer=opt, loss='categorical_hinge', metrics=['acc', 'mse'])

model.fit(X_train, y_train, epochs=70, batch_size=5)


Epoch 1/70
140/140 [==============================] - 0s 580us/step - loss: 0.9760 - acc: 0.6729 - mse: 0.2495
Epoch 2/70
140/140 [==============================] - 0s 557us/step - loss: 0.7788 - acc: 0.8743 - mse: 0.2136
Epoch 3/70
140/140 [==============================] - 0s 565us/step - loss: 0.3854 - acc: 0.9329 - mse: 0.0627
Epoch 4/70
140/140 [==============================] - 0s 564us/step - loss: 0.1153 - acc: 0.9971 - mse: 0.0065
Epoch 5/70
140/140 [==============================] - 0s 544us/step - loss: 0.0049 - acc: 1.0000 - mse: 5.6203e-05
Epoch 6/70
140/140 [==============================] - 0s 571us/step - loss: 0.0013 - acc: 1.0000 - mse: 8.6599e-06
Epoch 7/70
140/140 [==============================] - 0s 586us/step - loss: 5.3817e-04 - acc: 1.0000 - mse: 1.9978e-06
Epoch 8/70
140/140 [==============================] - 0s 596us/step - loss: 7.7775e-04 - acc: 1.0000 - mse: 4.8168e-06
Epoch 9/70
140/140 [==============================] - 0s 551us/step - loss: 5.0043e-04 -

In [ ]:
pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
# print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

def compareResult(predicted, real):
  print("Real: " + str(real) + " Predicted: " + str(predicted))

for i in range(10):
  compareResult(pred_train[i], y_train[i])

Accuracy on test data: 1.0% 
 Error on test data: 0.0
Real: [0. 0. 0. 0. 0. 0. 1. 0.] Predicted: [0. 0. 0. 0. 0. 0. 1. 0.]
Real: [0. 1. 0. 0. 0. 0. 0. 0.] Predicted: [0. 1. 0. 0. 0. 0. 0. 0.]
Real: [0. 0. 0. 0. 1. 0. 0. 0.] Predicted: [0. 0. 0. 0. 1. 0. 0. 0.]
Real: [0. 0. 1. 0. 0. 0. 0. 0.] Predicted: [0. 0. 1. 0. 0. 0. 0. 0.]
Real: [0. 1. 0. 0. 0. 0. 0. 0.] Predicted: [0. 1. 0. 0. 0. 0. 0. 0.]
Real: [0. 0. 1. 0. 0. 0. 0. 0.] Predicted: [0. 0. 1. 0. 0. 0. 0. 0.]
Real: [0. 0. 0. 0. 1. 0. 0. 0.] Predicted: [0. 0. 0. 0. 1. 0. 0. 0.]
Real: [0. 0. 0. 0. 0. 1. 0. 0.] Predicted: [0. 0. 0. 0. 0. 1. 0. 0.]
Real: [0. 0. 0. 1. 0. 0. 0. 0.] Predicted: [0. 0. 0. 1. 0. 0. 0. 0.]
Real: [0. 0. 1. 0. 0. 0. 0. 0.] Predicted: [0. 0. 1. 0. 0. 0. 0. 0.]


In [ ]:
for i in range(len(pred_test)):
  compareResult(pred_test[i], y_test[i])

Real: [0. 0. 0. 1. 0. 0. 0. 0.] Predicted: [0. 0. 0. 1. 0. 0. 0. 0.]
Real: [0. 0. 0. 0. 0. 0. 0. 1.] Predicted: [0. 0. 0. 0. 0. 0. 0. 1.]
Real: [0. 0. 0. 0. 0. 0. 1. 0.] Predicted: [0. 0. 0. 0. 0. 0. 1. 0.]
Real: [0. 0. 0. 0. 0. 0. 1. 0.] Predicted: [0. 0. 0. 0. 0. 0. 1. 0.]
Real: [0. 0. 0. 0. 0. 1. 0. 0.] Predicted: [0. 0. 0. 0. 0. 1. 0. 0.]
Real: [0. 0. 0. 0. 0. 0. 0. 1.] Predicted: [0. 0. 0. 0. 0. 0. 0. 1.]
Real: [1. 0. 0. 0. 0. 0. 0. 0.] Predicted: [1. 0. 0. 0. 0. 0. 0. 0.]
Real: [0. 0. 0. 0. 0. 0. 1. 0.] Predicted: [0. 0. 0. 0. 0. 0. 1. 0.]
Real: [0. 1. 0. 0. 0. 0. 0. 0.] Predicted: [0. 1. 0. 0. 0. 0. 0. 0.]
Real: [1. 0. 0. 0. 0. 0. 0. 0.] Predicted: [1. 0. 0. 0. 0. 0. 0. 0.]
Real: [1. 0. 0. 0. 0. 0. 0. 0.] Predicted: [1. 0. 0. 0. 0. 0. 0. 0.]
Real: [0. 0. 1. 0. 0. 0. 0. 0.] Predicted: [0. 0. 1. 0. 0. 0. 0. 0.]
Real: [0. 0. 0. 0. 1. 0. 0. 0.] Predicted: [0. 0. 0. 0. 1. 0. 0. 0.]
Real: [1. 0. 0. 0. 0. 0. 0. 0.] Predicted: [1. 0. 0. 0. 0. 0. 0. 0.]
Real: [1. 0. 0. 0. 0. 0. 0. 0.] Pr